In [12]:
import pandas as pd
import psycopg2
import requests

In [13]:
# Scripts de creacion de tablas

sql_especies = """ CREATE TABLE IF NOT EXISTS fmorosini_coderhouse.especies (
	id int NOT NULL,
	nombrevulgar varchar(50) NOT NULL,
	nombrecientifico varchar(50) NOT NULL,
	imagen varchar(50) NOT NULL,
	magnitud int4 NOT NULL,
	tipo varchar(15) NOT NULL,
	follaje varchar(15) NOT NULL,
	url_ficha varchar(255) NULL,
	thumbnail varchar(100) NULL,
	CONSTRAINT especies_pkey PRIMARY KEY (id)
); """


sql_localidades = """ CREATE TABLE IF NOT EXISTS fmorosini_coderhouse.localidades (
	id int NOT NULL,
	nombre varchar(80) NULL,
	CONSTRAINT localidades_pk PRIMARY KEY (id)
); """

sql_arboles = """ CREATE TABLE IF NOT EXISTS fmorosini_coderhouse.arboles (
	id int4 NOT NULL,
	lat float,
	lon float,
	especie int4 NOT NULL,
	localidad int4 NOT NULL,
	CONSTRAINT clave PRIMARY KEY (id)
); """

sql_FK_1 = """ ALTER TABLE fmorosini_coderhouse.arboles ADD CONSTRAINT "FK_especie" FOREIGN KEY (especie) REFERENCES fmorosini_coderhouse.especies(id); """
sql_FK_2 = """ ALTER TABLE fmorosini_coderhouse.arboles ADD CONSTRAINT "FK_localidad" FOREIGN KEY (localidad) REFERENCES fmorosini_coderhouse.localidades(id); """

In [14]:
# Defino connectionstring y abro conexión

con = psycopg2.connect("dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port= 5439 user=fmorosini_coderhouse password= 77Vz3W9KPP")

In [15]:
# Corro los sql de creación de tablas y Foreign Keys

cursor = con.cursor()

cursor.execute(sql_especies)
cursor.execute(sql_localidades)
cursor.execute(sql_arboles)
con.commit()

try:
    cursor.execute(sql_FK_1)
    cursor.execute(sql_FK_2)
    con.commit()
except:
    print("Error, las FK ya están creadas")
    con.rollback()

con.close()

Error, las FK ya están creadas


In [16]:
# URLs de endpoints de las 3 entidades

url_especies = "https://api.arbolesurbanos.com.ar/json/especies"
url_localidades = "https://api.arbolesurbanos.com.ar/json/localidades"
url_arboles = "https://api.arbolesurbanos.com.ar/json/arboles"

# Traigo la data

req_especies = requests.get(url_especies)
req_localidades = requests.get(url_localidades)
req_arboles = requests.get(url_arboles)

# Convierto a Dict

especies = req_especies.json()["data"]
localidades = req_localidades.json()["data"]
arboles = req_arboles.json()["data"]

# Cargo DataFrames con los dict

especies_df = pd.DataFrame.from_dict(especies)
localidades_df = pd.DataFrame.from_dict(localidades)

In [17]:
# Al de arboles hay que laburarlo un poco más porque está en GeoJson, un Json específico para guardar datos georreferenciados

arboles_list = []

for arbol in arboles:

    arbol_dict = {}

    arbol_dict["id"] = arbol["id"]
    arbol_dict["localidad"] = arbol["localidad"]
    arbol_dict["especie"] = arbol["especie"]
    arbol_dict["lon"] = arbol["posicion"]["coordinates"][0]
    arbol_dict["lat"] = arbol["posicion"]["coordinates"][1]

    arboles_list.append(arbol_dict)

In [18]:
arboles_df = pd.DataFrame.from_dict(arboles_list)

In [19]:
# Vuelvo a abrir conexión para eliminar datos de tablas

con = psycopg2.connect("dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port= 5439 user=fmorosini_coderhouse password= 77Vz3W9KPP")

cursor = con.cursor()

In [20]:
sql_delete_arboles = """ DELETE FROM arboles WHERE 1 = 1 """
sql_delete_especies  = """ DELETE FROM especies WHERE 1 = 1 """
sql_delete_localidades = """ DELETE FROM localidades WHERE 1 = 1 """

cursor.execute(sql_delete_arboles)
cursor.execute(sql_delete_especies)
cursor.execute(sql_delete_localidades)

con.commit()

con.close()

In [ ]:
con = psycopg2.connect("dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port= 5439 user=fmorosini_coderhouse password= 77Vz3W9KPP")

def inserta_localidades(fila):

    cursor = con.cursor()

    cursor.execute(""" INSERT INTO localidades (id,nombre) VALUES (%s,%s)""",(fila["ogc_fid"],fila["nombre"]))

    con.commit()

def inserta_especies(fila):

    cursor = con.cursor()

    cursor.execute(""" INSERT INTO especies (id,nombrevulgar,nombrecientifico,imagen,magnitud,tipo,follaje,url_ficha,thumbnail) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)""",(fila["id"],fila["nombrevulgar"],fila["nombrecientifico"],fila["imagen"],fila["magnitud"],fila["tipo"],fila["follaje"],fila["url_ficha"],fila["thumbnail"]))

    con.commit()

def inserta_arboles(fila):

    cursor = con.cursor()

    cursor.execute(""" INSERT INTO arboles (id,especie,localidad,lat,lon) VALUES (%s,%s,%s,%s,%s)""",(fila["id"],fila["especie"],fila["localidad"],fila["lat"],fila["lon"]))

    con.commit()
    

localidades_df.apply(inserta_localidades,axis=1)
especies_df.apply(inserta_especies,axis=1)

#Solo inserto 100 árboles porque son 12000 y pico y tarda demasiado

arboles_df.head(100).apply(inserta_arboles,axis=1)

con.close()

In [ ]:
# Limpio memoria

del arboles_df
del localidades_df
del especies_df